In [79]:
import scipy.io
import os

mat = scipy.io.loadmat("/Users/ericschreiber/Downloads/training2017/A04953.mat")

In [80]:
for i in range(30):
    # print on the same line with ,
    print(mat["val"][0][i], end=",")

96,114,137,167,185,195,202,207,210,212,213,215,218,220,223,224,220,212,203,194,184,172,158,142,127,112,98,88,80,72,

In [27]:
import sys
import tqdm

sys.path.append(".")
from util import load_base_data


X_train, y_train, X_test = load_base_data()

In [57]:
import hashlib
import numpy as np


def make_fingerprint(ecg):
    # ecg is a numpy array
    # calculate the fingerprint using a hash function
    # return the fingerprint
    # return hashlib.sha256(ecg.tobytes()).hexdigest()
    # return the sum of the ecg, nan values shoul be ignored
    sum = 0
    for i in range(len(ecg)):
        multiplikator = i % 13
        if not np.isnan(ecg[i]):
            sum += ecg[i] * multiplikator
    return sum


print(make_fingerprint(X_train.iloc[0].values))
print(make_fingerprint(mat["val"][0]))

867568.0
1438253


In [58]:
def fingerprints_aml_data(X_train, X_test):
    # X_train and X_test are pandas dataframes
    # return sets of fingerprints and their corresponding indices
    # in the original dataframes
    train_fingerprints = {}
    test_fingerprints = {}

    for i in tqdm.tqdm(range(len(X_train))):
        train_fingerprints[make_fingerprint(X_train.iloc[i].values)] = i

    for i in tqdm.tqdm(range(len(X_test))):
        test_fingerprints[make_fingerprint(X_test.iloc[i].values)] = i

    return train_fingerprints, test_fingerprints


train_fingerprints, test_fingerprints = fingerprints_aml_data(X_train, X_test)

100%|██████████| 3411/3411 [00:47<00:00, 72.39it/s]


In [59]:
# check if the dictionaries have the same size as the original dataframes
assert len(train_fingerprints) == len(X_train)
assert len(test_fingerprints) == len(X_test)

AssertionError: 

In [81]:
def fingerprints_from_physionet(path):
    # for all mat files in the given path
    # 1. load the mat file
    # 2. calculate the fingerprint
    # 3. store the fingerprint and the corresponding filename in a list
    # return the list of fingerprints and the list of filenames
    fingerprints = []

    for file in tqdm.tqdm(os.listdir(path)):
        if file.endswith(".mat"):
            mat = scipy.io.loadmat(os.path.join(path, file))
            fingerprints.append((make_fingerprint(mat["val"][0]), file))

    return fingerprints


physionet_fingerprints = fingerprints_from_physionet(
    "/Users/ericschreiber/Downloads/training2017"
)
physionet_samples_fingerprints = fingerprints_from_physionet(
    "/Users/ericschreiber/Downloads/sample2017/validation"
)

100%|██████████| 602/602 [00:04<00:00, 138.40it/s]


In [69]:
# find all fingerprints that are in both datasets
# and store them in a list
common_fingerprints_train = []
common_fingerprints_test = []

for fingerprint in tqdm.tqdm(physionet_fingerprints):
    if fingerprint[0] in train_fingerprints:
        common_fingerprints_train.append(
            (fingerprint[0], fingerprint[1], train_fingerprints[fingerprint[0]])
        )
    if fingerprint[0] in test_fingerprints:
        common_fingerprints_test.append(
            (fingerprint[0], fingerprint[1], test_fingerprints[fingerprint[0]])
        )

100%|██████████| 8528/8528 [00:00<00:00, 2238361.98it/s]


In [70]:
common_fingerprints_train

[(-218070, 'A07719.mat', 608),
 (911114, 'A07635.mat', 4967),
 (644683, 'A05625.mat', 1678),
 (1111079, 'A04870.mat', 893),
 (1018827, 'A06323.mat', 22),
 (-408636, 'A01850.mat', 2148),
 (1800218, 'A05088.mat', 68),
 (23118, 'A05274.mat', 3953)]

In [71]:
common_fingerprints_test

[(427928, 'A03232.mat', 3021),
 (-152913, 'A04953.mat', 2961),
 (732431, 'A05479.mat', 145),
 (1377513, 'A04885.mat', 1440),
 (1443217, 'A04514.mat', 2736),
 (1102743, 'A03874.mat', 73),
 (724529, 'A06142.mat', 2385),
 (1054789, 'A06620.mat', 452),
 (543578, 'A02269.mat', 1228),
 (1918995, 'A02057.mat', 2742)]

In [83]:
# find all fingerprints that are in both datasets
# and store them in a list
common_fingerprints_train_samples = []
common_fingerprints_test_samples = []

for fingerprint in tqdm.tqdm(physionet_samples_fingerprints):
    if fingerprint[0] in train_fingerprints:
        common_fingerprints_train_samples.append(
            (fingerprint[0], fingerprint[1], train_fingerprints[fingerprint[0]])
        )
    if fingerprint[0] in test_fingerprints:
        common_fingerprints_test_samples.append(
            (fingerprint[0], fingerprint[1], test_fingerprints[fingerprint[0]])
        )

print(len(common_fingerprints_train_samples))
print(len(common_fingerprints_test_samples))

100%|██████████| 300/300 [00:00<00:00, 954697.42it/s]

0
0
